In [1]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 7, 1, Finished, Available)

In [2]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("RacaVitima", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 7, 2, Finished, Available)

In [3]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Raça-cor da Vítima.csv')

StatementMeta(tcccibele, 7, 3, Finished, Available)

In [4]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 7, 4, Finished, Available)

In [5]:
# Remover "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'RacaVitima':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 7, 5, Finished, Available)

In [6]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/RacaVitima/')

StatementMeta(tcccibele, 7, 6, Finished, Available)

In [7]:
df = spark.read.parquet('/history_zone/2019/RacaVitima/')
df.toPandas()

StatementMeta(tcccibele, 7, 7, Finished, Available)

,RacaVitima,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,Amarela,97,79,88,74,62,66,65,67,52,65,48,29
1,Branca,3449,2874,3135,2632,2726,2434,2394,2840,2769,2718,2585,1332
2,Indígena,62,43,57,50,45,42,64,40,40,37,25,21
3,Parda,4084,3235,3530,3171,3198,2717,2866,3376,3236,3253,2924,1491
4,Preta,1070,876,1011,916,894,748,783,922,868,822,729,457
5,Não Informado,787,562,619,400,384,293,270,383,347,385,377,222
